In [20]:
import numpy as np
from hmmlearn import hmm
import pandas as pd


In [23]:
# defining matrixes
np.random.seed(42)
components = 6
delta = 0.05
m = 5
startprob = np.array([1, 0, 0, 0, 0, 0])
transtprob = np.array([[0.1, 0.9, 0, 0, 0, 0],
                        [0.2, 0, 0.1+delta, 0, 0, 0.7-delta],
                        [0, 0.5, 0, 0.5, 0, 0],
                        [0, 0, 0.5, 0.5, 0, 0],
                        [0, 0, 0.1, 0, 0.4, 0.5],
                        [0, 0.2, 0, 0, 0.7, 0.1]])
emissionprob = np.array([[0.25, 0.25, 0.25, 0.25],
                         [0.4, 0.3, 0.2, 0.1],
                         [0.4, 0.4, 0.1, 0.1],
                         [0.3, 0.3, 0.3, 0.1],
                         [0.2, 0.2, 0.2, 0.4],
                         [0.2, 0.2, 0.3, 0.3]
                         ])


In [11]:
# model instatiation
model = hmm.CategoricalHMM(n_components=components)
model.startprob_=startprob
model.transmat_ = transtprob
model.emissionprob_ = emissionprob

In [12]:
def observation_sample(model_gen, k = 10):
    O, Q = model_gen.sample(k*2)
    observations = O[np.arange(0,2*k,2)].flatten()
    return observations, Q

In [13]:
obs, real_states= observation_sample(model_gen=model, k=200)

In [16]:
# definition of Viterbi alg with obs every other state
def viterbi(obs, components):
    T = len(obs)*2
    V = np.zeros((T,componets))
    for j in range(componets):
        V[0][j] = startprob[j]*emissionprob[j][obs[0]]
    for t in range(1,T):
        for j in range(components):
            if t%2 == 0:
                V[t][j] = max(V[t-1] * transtprob[:,j] * emissionprob[j][obs[int(t/2)]])
            else:
                V[t][j] = max(V[t-1] * transtprob[:,j])
    optimal_state = np.argmax(V[T-1])
    best_path = [optimal_state]
    for i in range(T-2,-1,-1):
        optimal_state = np.argmax(V[i-1]*transtprob[:,optimal_state])
        best_path.append(optimal_state)
    return best_path

In [18]:
def high_states_check(path, m, high_states=[4,5]):
    path_np = np.array(path)
    path_np[path_np>=min(high_states)] = 7
    for i in range(len(path) - m):
        if np.array_equal(path_np[i:i+m], np.array([7]*m)):
            # print("Fuori controllo")
            return True
    return False

In [19]:
high_states_check(viterbi(obs=obs,components=components), m=5)

True

In [21]:
def high_risk_condition(model, components, m, k,n, f):
    cons_state = []
    for i in range(n):
        obs, real_states= observation_sample(model_gen=model, k=k)
        cons_state.append(high_states_check(viterbi(obs=obs,components=components), m=m))
    return cons_state

In [25]:
# definition of parameters
m = 5
k = 200
n = 100
f = 0.1

In [26]:
states_delta05 = high_risk_condition(model=model, components=components, m=m,k=k,n=n,f=f)

In [27]:
states_delta05

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [34]:
# defining matrixes
np.random.seed(42)
components = 6
delta = 0.5
m = 5
startprob = np.array([1, 0, 0, 0, 0, 0])
transtprob = np.array([[0.1, 0.9, 0, 0, 0, 0],
                        [0.2, 0, 0.1+delta, 0, 0, 0.7-delta],
                        [0, 0.5, 0, 0.5, 0, 0],
                        [0, 0, 0.5, 0.5, 0, 0],
                        [0, 0, 0.1, 0, 0.4, 0.5],
                        [0, 0.2, 0, 0, 0.7, 0.1]])
emissionprob = np.array([[0.25, 0.25, 0.25, 0.25],
                         [0.4, 0.3, 0.2, 0.1],
                         [0.4, 0.4, 0.1, 0.1],
                         [0.3, 0.3, 0.3, 0.1],
                         [0.2, 0.2, 0.2, 0.4],
                         [0.2, 0.2, 0.3, 0.3]
                         ])


In [35]:
states_delta2 = high_risk_condition(model=model, components=components, m=m,k=k,n=n,f=f)

In [36]:
states_delta2

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]